## <center><strong>Metode Numerik</strong><br />
<img alt="" src="images/MetNum.png" style="height: 200px;" /></center>

# <center><font color="blue">Turunan Numerik</font></center>

<h2 id="(C)-Taufik-Sutanto---2019" style="text-align: center;">(C) Taufik Sutanto - 2019</h2>
<h2 id="tau-data-Indonesia-~-https://tau-data.id" style="text-align: center;">tau-data Indonesia ~ <a href="https://tau-data.id" target="_blank"><span style="color: #0009ff;">https://tau-data.id</span></a></h2>

# Notes and Disclaimer

* This notebook is part of the free (open knowledge) eLearning course at: https://tau-data.id/courses/
* Some images are taken from several resources, we respect those images ownerships and put a reference/citation from where it is originated. Nevertheless, sometimes we are having trouble to find the origin of the image(s). If you are the owner of the image and would like the image taken-out (or want the citation to be revised) from this open knowledge course resources please contact us here with the details: https://tau-data.id/contact/  
* Unless stated otherwise, in general tau-data permit its resources to be copied and-or modified for non-commercial purposes. With condition proper acknowledgement/citation is given.

## Turunan Numerik:

* Pendahuluan
* Pendekatan Turunan Numerik 
* Analisis Error
* Ekstrapolasi Richardson
* Formula-formula turunan numerik

## Sebelum dimulai
* Yang akan dibahas di Bab ini hanya pendekatan dari turunan "sebuah fungsi" di suatu titik
* Sistem "Ordinary Differential Equations" / Persamaan Differensial Biasa akan dibahas di Buku di Bab IX.

# Ketika Limit Fungsi Real di Kalkulus Ternyata Bukan Dari Dunia Nyata

* Dari Kalkulus kita mendapatkan bahwa $$f'(x) = \lim_{h\rightarrow 0}\frac{f(x+h)-f(x)}{h}$$
* Secara logika kita memahami limit diatas sebagai 
  "*ketika h semakin dekat dengan 0, maka nilai limit fungsinya semakin mendekati kemiringan garis singgung fungsi f di x*"
* Permasalahannya komputer tidak bisa menggunakan (menyimpan) bilangan real.
* Apakah pengertian limit diatas berlaku juga di sistem Floating Point?

* Misal $$f(x) = e^x$$
* Maka $f'(1) = e = 2.718281828459045 ...$
* Mari kita hitung $$\frac{f(x+h)-f(x)}{h}$$ untuk $h=10^{-1},10^{-2},...,10^{-10},...$

In [7]:
# Contoh Hal 312
import numpy as np

def f(x):
    return x*np.exp(x)

X = 0 # f'(0)
H = [10**-i for i in range(1,15)] # [0.1, 0.01, 0.001, 0.0001, 1e-05, 1e-06, 1e-07, 1e-08, 1e-09]
H = [0.1]
limit = [(f(X+h)-f(X))/h for h in H]
E = [abs(f(X)-l) for l in limit]
print ('h\t\t\tlimit\t\t\t    error_Absolut')
for h, l, e in zip(H,limit, E):
    print(h,'\t\t',l,'\t\t',e)
    
[abs(1-l) for l in limit]

h			limit			    error_Absolut
0.1 		 1.1051709180756477 		 1.1051709180756477


[0.10517091807564771]

# Komputer 
* Pada contoh diatas kita melihat bahwa **h** mengecil belum tentu errornya juga mengecil
* Kita bisa mengira-ngira **h terbaik** pada kasus diatas, tapi bagaimana dengan fungsi yang lain?

<h2 id="Central-Difference">Central Difference</h2>

<p><img alt="" src="images/theo_central_difference.png" style="width: 800px; height: 424px;" /></p>


In [2]:
# Better?
limit = [(f(X+h)-f(X-h))/(2*h) for h in H]
E = [abs(f(X)-l) for l in limit]
print ('h\t\t\tlimit\t\t\t    error_Absolut')
for h, l, e in zip(H,limit, E):
    print(h,'\t\t',l,'\t\t',e)

h			limit			    error_Absolut
0.1 		 2.7228145639474177 		 0.00453273548837263
0.01 		 2.718327133382714 		 4.530492366905392e-05
0.001 		 2.718282281505724 		 4.53046678838831e-07
0.0001 		 2.718281832989611 		 4.53056570037802e-09
1e-05 		 2.718281828517632 		 5.858691309867936e-11
1e-06 		 2.718281828295588 		 1.634572477371421e-10
1e-07 		 2.7182818285176324 		 5.858735718788921e-11
1e-08 		 2.7182818218562943 		 6.60275079056305e-09
1e-09 		 2.7182818218562943 		 6.60275079056305e-09
1e-10 		 2.7182811557224795 		 6.72736565565657e-07
1e-11 		 2.7182922579527258 		 1.0429493680685908e-05
1e-12 		 2.7184920980971583 		 0.00021026963811321409
1e-13 		 2.717825964282383 		 0.00045586417666187984
1e-14 		 2.708944180085382 		 0.009337648373663132


<h2 id="Taking-the-central-Difference-Further">Taking the central Difference Further</h2>

<p><img alt="" src="images/Theo_Central_Difference_2.png" style="width: 800px; height: 378px;" /></p>


In [2]:
# Really?
limit = [(-f(X+2*h)+8*f(X+h)-8*f(X-h)+f(X-2*h))/(12*h) for h in H]
E = [abs(f(X)-l) for l in limit]
print ('h\t\t\tlimit\t\t\t    error_Absolut')
for h, l, e in zip(H,limit, E):
    print(h,'\t\t',l,'\t\t',e)
([abs(0-l) for l in limit][0])

h			limit			    error_Absolut
0.1 		 0.0 		 0.0


10.517091807564771

## Wow that's almost magical! It's time we understand where are these formulas came from.

* Derive Taylor expansion $f(x) = P_4(x)+E_4(x)$ untuk f(x+h) dan f(x-h)
* $f(x+h) - f(x-h) = 2f'(x)h + 2f^{(3)}(x)h^3/3! + 2f^{(5)}(c_1)h^5/5!$
* Lakukan hal yang sama, tapi h = 2h, sehingga
* $f(x+2h) - f(x-2h) = 4f'(x)h + 16f^{(3)}(x)h^3/3! + 64f^{(5)}(c_2)h^5/5!$
* Persamaan kedua dikurangi 8 kali persamaan pertama menghasilkan formula diatas. 
* $-f(x+2h) + 8f(x+h) - 8f(x-h) + f(x-2h) = 12 f'(x)h + \frac{(16f^{(5)}(c_1) - 64 f^{(5)}(c_2))h^5}{120}$
* Jika dimisalkan 
* $16 f^{(5)}(c_1) - 64 f^{(5)}(c_2) = -48f^{(5)}(c)$ untuk suatu $c\in [x-2h, x+2h]$, maka
* $f'(x) = \frac{-f(x+2h) + 8 f(x+h) -8 f(x-h) + f(x-2h)}{12h} + \frac{f^{(5)}(c)h^4}{30}$
* Hence our previous formula

<h2 id="Analisis-Error,-Teori-dan-Realitas">Analisis Error, Teori dan Realitas</h2>

<ul>
	<li>Jika dibaca di buku, maka kita mendapatkan secara analitik batas error dari teorema 6.1 adalah:</li>
	<li><img alt="" src="images/cor_6.1.b.png" style="width: 400px; height: 142px;" /></li>
	<li>dan untuk teorema 6.2 adalah:</li>
	<li><img alt="" src="images/cor_6.2.b.png" style="width: 400px ; height: 141px" /></li>
	<li>But how useful are these corollary? Why? to what extend?</li>
</ul>


<h2 id="Richardson-Extrapolation">Richardson Extrapolation</h2>

<ul>
	<li>What Extrapolation? What do we mean by extrapolation in this contex?</li>
	<li><img alt="" src="images/richard_extrapolation.png" style="width: 600px; height: 256px;" /></li>
	<li>How do we use it?</li>
</ul>


<p><img alt="" src="images/richard_extrapolation_alg.png" style="width: 624px; height: 172px;" /></p>


In [4]:
# First you need to understand about List indexing in Python
A = [3, 4, 5, 6, 9, 8, 7]
print(A[-1], A[-2], A[-3])

7 8 9


In [8]:
# Contoh aplikasi
X = 0
h = 0.1 # h terbaik di contoh terakhir
D = []
# Lihat formulanya, kita butuh minimal 3 D untuk memulai iterasinya.
# Kita masih akan menggunakan f(x) yang sama dengan contoh sebelumnya

def R(x,h,k):
    return (f(x+10**-k*h) - f(x-10**-k*h))/(2*h*10**-k)

print ('k\t\t\tD\t\t\t    error_Absolut')

k = 0
d = R(X,h,k) # k = 0
D.append(d)
print(k,'\t\t', d,'\t\t',abs(1-d))

k = k+1
d = R(X,h,k) # k = 1
D.append(d)
print(k,'\t\t', d,'\t\t',abs(1-d))

k = k+1
d = R(X,h,k) # k = 2
D.append(d)
print(k,'\t\t', d,'\t\t',abs(1-d))

while abs(D[-1]-D[-2])<abs(D[-2]-D[-3]):
    k = k+1
    d = R(X,h,k)
    D.append(d)
    print(k,'\t\t', d,'\t\t',abs(1-d))
    
print('Final solution at k = ', k-2)

k			D			    error_Absolut
0 		 1.0050041680558035 		 0.005004168055803504
1 		 1.0000500004166681 		 5.0000416668138925e-05
2 		 1.0000005000000418 		 5.000000418142747e-07
3 		 1.0000000050000002 		 5.00000019165725e-09
4 		 1.0000000000499998 		 4.9999782092413625e-11
5 		 1.0000000000005 		 5.000444502911705e-13
6 		 1.000000000000005 		 5.10702591327572e-15
7 		 1.0 		 0.0
8 		 1.0000000000000002 		 2.220446049250313e-16
9 		 1.0 		 0.0
Final solution at k =  7


<h2 id="Numerical-Differentiation-Formulas">Numerical Differentiation Formulas</h2>

<p><img alt="" src="images/NDF_1.png" style="width: 462px; height: 538px;" /></p>


<p><img alt="" src="images/NDF_2.png" style="width: 563px; height: 548px;" /></p>


# End of Module

<hr />